# Export and convert saved model to tflite model TF1


## Workspace preparation

In [1]:
%tensorflow_version 1.x

%cd /content/

! git clone --depth 1 https://github.com/tensorflow/models
! pip install -q tf_slim
! pip install -qq pycocotools

%cd /content/models/research/
! protoc object_detection/protos/*.proto --python_out=.

from google.colab import drive
drive.mount('/content/drive')
!rm -rf /content/sample_data

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/:/content/models/research/slim/'

%cd /content/drive/My Drive/projects/ML/object_detection_tf/workspace

TensorFlow 1.x selected.
/content
Cloning into 'models'...
remote: Enumerating objects: 2232, done.
remote: Counting objects: 100% (2232/2232), done.
remote: Compressing objects: 100% (1930/1930), done.
remote: Total 2232 (delta 541), reused 976 (delta 277), pack-reused 0
Receiving objects: 100% (2232/2232), 30.48 MiB | 30.13 MiB/s, done.
Resolving deltas: 100% (541/541), done.
     |████████████████████████████████| 358kB 2.8MB/s 
/content/models/research
Mounted at /content/drive
/content/drive/My Drive/projects/ML/object_detection_tf/workspace


## Model selection

In [2]:
selected_model = "faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28" #@param ["ssd_mobilenet_v2_quantized_300x300", "ssd_mobilenet_v3_small_coco", "ssdlite_mobilenet_edgetpu_coco_quant", "faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28"]
pipeline_path = f'my_models/pipelines/{selected_model}.config'
checkpoint_steps = 500 #@param {type:"number"}
trained_checkpoint_prefix = f'my_models/{selected_model}/model.ckpt-{checkpoint_steps}'
exported_models_dir = f'exported_models/{selected_model}/'

## Exporting model

### Export inference graph

In [3]:
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path {pipeline_path} \
    --trained_checkpoint_prefix {trained_checkpoint_prefix} \
    --output_directory {exported_models_dir}


INFO:tensorflow:Scale of 0 disables regularizer.
I1008 22:30:32.500048 139831445743488 regularizers.py:99] Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
I1008 22:30:32.500346 139831445743488 regularizers.py:99] Scale of 0 disables regularizer.
Instructions for updating:
Please use `layer.__call__` method instead.
W1008 22:30:32.501384 139831445743488 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:Scale of 0 disables regularizer.
I1008 22:30:41.816339 139831445743488 regularizers.py:99] Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
I1008 22:30:42.079090 139831445743488 regularizers.py:99] Scale of 0 disables regularizer.
INFO:tensorflow:depth of additional conv before b

### Export TFlite SSD graph

In [ ]:
%cd {PROJECT_ROOT_PATH + "/workspace"}

!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path={pipeline_path} \
    --trained_checkpoint_prefix={trained_checkpoint_prefix} \
    --output_directory={exported_models_dir} \
    --add_postprocessing_op=true


## Convert to tflite model

#### Command line *** working with Edge ***

In [ ]:
output_dir = f'{exported_models_dir}/detect.tflite'
graph = f'{exported_models_dir}tflite_graph.pb'

!tflite_convert \
  --graph_def_file={graph} \
  --output_file={output_dir} \
  --output_format=TFLITE \
  --input_shapes=1,320,320,3 \
  --input_arrays=normalized_input_image_tensor \
  --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
  --inference_type=QUANTIZED_UINT8 \
  --mean_values=128 \
  --std_dev_values=128 \
  --change_concat_input_ranges=False \
  --allow_custom_ops


#### Python API

In [ ]:
import tensorflow as tf

# Convert the model.
converter = tf.lite.TFLiteConverter.from_frozen_graph(
    graph_def_file=f'{exported_models_dir}tflite_graph.pb',
    input_arrays=['normalized_input_image_tensor'],
    input_shapes={'normalized_input_image_tensor' : [1, 320, 320,3]},
    output_arrays=['TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3']
)
converter.inference_type = tf.uint8
converter.quantized_input_stats = {'normalized_input_image_tensor': [128, 127]}
converter.allow_custom_ops = True
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()

# Save the model.
with open(f'{exported_models_dir}/detect.tflite', 'wb') as f:
  f.write(tflite_model)